# PyTorch Modelini İleri Seviyeye Geliştirme (Advanced Enhancements)
Bu kısımda CleanEnhancedANN modelimize ekleyebileceğimiz gelişmiş özellikleri adım adım inceleyeceğiz.


---



## 1️⃣ Farklı Aktivasyon Fonksiyonları

Aktivasyon fonksiyonları, nöronların çıkış değerlerini dönüştürerek modelin **karmaşık ilişkileri öğrenmesini** sağlar.  
Şu an modelimizde `LeakyReLU` kullanıyoruz, ama başka seçenekler de var:

```python
# ReLU
relu = nn.ReLU()  
# LeakyReLU
leaky_relu = nn.LeakyReLU(0.1)  
# ELU
elu = nn.ELU(alpha=1.0)  
# SELU
selu = nn.SELU()  
# Tanh
tanh = nn.Tanh()  
# Sigmoid
sigmoid = nn.Sigmoid()



##  Aktivasyon Fonksiyonları – Kod ve Açıklama Tablosu

| Aktivasyon | Kod | Açıklama |
|------------|-----|----------|
| ReLU | `nn.ReLU()` | Çıkış = max(0, x). Basit ve hızlı, derin ağlarda yaygın. Dezavantaj: negatiflerde ölü nöron problemi. |
| LeakyReLU | `nn.LeakyReLU(0.1)` | Çıkış = x (x>0), 0.1*x (x<0). Negatiflerde küçük gradyan, ölü nöron sorununu azaltır. Mevcut modelde kullanılıyor. |
| ELU | `nn.ELU(alpha=1.0)` | x>0 → x, x<=0 → alpha*(exp(x)-1). Negatif değerlerde gradyan verir, öğrenme stabilitesini artırır. Derin ağlarda ReLU’dan daha iyi performans gösterebilir. |
| SELU | `nn.SELU()` | Kendiliğinden normalizasyon sağlar (self-normalizing). Derin ağlarda gradyan kaybını ve patlamasını azaltır. Özellikle katman sayısı fazla modeller için güçlü. |
| Tanh | `nn.Tanh()` | Çıkış aralığı [-1, 1]. Sinyali normalize eder. Dezavantaj: çok derin ağlarda vanishing gradient problemi yaşanabilir. |
| Sigmoid | `nn.Sigmoid()` | Çıkış aralığı [0,1]. Genellikle çıkış katmanı için kullanılır (binary classification). Dezavantaj: çok derin katmanlarda gradyan kaybı olabilir. |


## 2️⃣ Aktivasyon Fonksiyonları – Kullanım Senaryoları

| Aktivasyon | Kullanım Durumu / İhtimal | Notlar |
|------------|---------------------------|--------|
| ReLU | Genel derin ağlar, hızlı ve basit modeller | Negatif değerleri sıfırladığı için ölü nöron problemi olabilir |
| LeakyReLU | Derin ağlar, ölü nöron riskini azaltmak istediğinde | Küçük negatif slope ile negatif gradyan sağlar, mevcut modelde kullanılıyor |
| ELU | Derin ağlar, daha stabil öğrenme ve negatif değerlerin önemli olduğu durumlar | ReLU’dan daha iyi performans gösterebilir, küçük negatif gradyan verir |
| SELU | Çok derin katmanlı ağlar, self-normalizing gereken durumlar | Otomatik normalize eder, gradyan kaybı/patlamasını azaltır |
| Tanh | Giriş ve çıkış değerlerini normalize etmek istediğinde, sinyaller [-1,1] aralığında olmalı | Vanishing gradient riski derin ağlarda yüksek |
| Sigmoid | Binary classification çıkış katmanı, olasılık tahmini | Derin katmanlarda gradyan kaybı olabilir, genellikle sadece çıkış katmanında kullanılır |


## 🔹 Aktivasyon Fonksiyonları – Ek Notlar ve İpuçları

1. **Çıkış Katmanı ile Uyumluluk**
   - `Sigmoid` → binary classification için çıkış katmanı
   - `Softmax` → multi-class classification için çıkış katmanı
   - Gizli katmanda Softmax kullanmak genellikle önerilmez

2. **Derinlik ve Gradyan Problemleri**
   - ReLU ve LeakyReLU → gradyan kaybını (vanishing gradient) azaltır
   - Tanh ve Sigmoid → derin ağlarda gradyan kaybına yol açabilir, dikkatli kullanılmalı

3. **Performans ve Hız**
   - ReLU → en hızlı, GPU’larda optimize edilmiş
   - ELU / SELU → biraz daha yavaş ama stabilite ve performans artışı sağlar

4. **Kendi Aktivasyonunu Oluşturma**
   - `nn.Module` veya `torch.autograd.Function` ile custom activation yazılabilir
   - Örnek: negatif değerleri özel bir şekilde ölçeklendirmek veya yeni non-linearity denemek

5. **Kombinasyonlar**
   - İleri modellerde birden fazla aktivasyon bir arada kullanılabilir
   - Örnek: ilk katman ReLU, ikinci katman ELU → stabilite ve hız dengesi sağlanabilir



💡 **Özet:**  
- Aktivasyon seçimi **gizli katman vs çıkış katmanı** ve **veri tipi / problem tipi** ile doğrudan ilişkili.  
- Çıkış katmanı için problem türüne uygun aktivasyon seçmek kritik.  
- Derinlik ve gradyan problemleri göz önünde bulundurulmalı.  
- İleri seviyede **custom veya kombinasyonlu aktivasyonlar** deneyebilirsin.


----

# 2️⃣ Residual (Skip) Bağlantıları ve Normalizasyon

Bu bölümde modelin stabilitesini ve öğrenme kapasitesini artıracak ileri seviye geliştirmeleri inceleyeceğiz.



## 🔹 Residual (Skip) Bağlantıları

- Amaç: Derin ağlarda **gradyan kaybını önlemek** ve öğrenmeyi hızlandırmak
- Mantık: Bir katmanın çıktısını bir sonraki katmana **direkt ekler**
- Örnek:

```python
class ResidualBlock(nn.Module):
    def __init__(self, size):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(size, size),
            nn.BatchNorm1d(size),
            nn.ReLU()
        )

    def forward(self, x):
        return x + self.block(x)


* Residual bağlantıları diğer konularda daha net anlayacağız.ANN için bu kadar örnek yeterlidir diye düşünüyorum.

## 🔹 Batch Normalization (BatchNorm)

- **Amaç:** Katman çıkışlarını normalize ederek eğitimi stabil ve hızlı hale getirir

- **Örnek:**
```python
bn = nn.BatchNorm1d(64)  # 64 nöronlu bir katman için
x = bn(x)


## 🔹 Layer Normalization (LayerNorm)

- **Amaç:** Küçük batch veya sequence verilerde stabilite sağlamak

- **Örnek:**
```python
ln = nn.LayerNorm(64)
x = ln(x)


Notlar:

* Batch boyutuna bağımlı değildir

* NLP ve sequence modellerinde sıklıkla tercih edilir

---
# 3️⃣ Weight Initialization ve Regularization

Bu bölümde modelin **öğrenme hızı, stabilitesi ve overfitting kontrolü** için kullanılabilecek teknikleri inceleyeceğiz.



## 🔹 Weight Initialization

- Amaç: Ağın ağırlıklarını uygun bir şekilde başlatarak **hızlı ve stabil öğrenme** sağlamak
- Örnekler:

```python
# Xavier uniform initialization
nn.init.xavier_uniform_(layer.weight)
nn.init.zeros_(layer.bias)

# Kaiming normal initialization (ReLU tabanlı ağlar için)
nn.init.kaiming_normal_(layer.weight, nonlinearity='relu')
nn.init.zeros_(layer.bias)


Notlar:

* Xavier → genellikle sigmoid / tanh tabanlı ağlarda iyi

* Kaiming → ReLU / LeakyReLU tabanlı ağlarda daha iyi performans

* Orthogonal initialization → derin ağlarda gradyan stabilitesi sağlar

## 🔹 DropConnect

Amaç: Dropout’a benzer ama ağırlıkları rastgele kapatır

Örnek:

In [ ]:
import torch.nn as nn
dropconnect = nn.Dropout(p=0.2)  # PyTorch'da DropConnect için custom layer yazılabilir

Notlar:

* Daha agresif regularization sağlar

* Küçük datasetlerde overfitting’i azaltmak için tercih edilebilir

## 🔹 Özet Tablosu

| Teknik                 | Amaç                                                  | Kullanım / Notlar                                |
| ---------------------- | ----------------------------------------------------- | ------------------------------------------------ |
| Xavier Initialization  | Stabil öğrenme, hızlı konverjans                      | Sigmoid / Tanh tabanlı ağlar                     |
| Kaiming Initialization | ReLU tabanlı ağlarda iyi performans                   | ReLU / LeakyReLU tabanlı ağlar                   |
| Dropout                | Overfitting’i azaltmak                                | Eğitim sırasında bazı nöronları rastgele kapatır |
| DropConnect            | Ağırlıkları rastgele kapatarak overfitting’i azaltmak | Özellikle küçük datasetlerde faydalı             |


---

# 4️⃣ Learning Rate ve Scheduler’lar

Learning Rate (LR) modelin öğrenme hızını belirler ve doğru ayarlanmazsa eğitim **çok yavaş veya kararsız** olabilir.  
Scheduler’lar, LR’yi dinamik olarak değiştirerek konverjansı hızlandırır ve stabil hale getirir.



## 🔹 Sabit Learning Rate
```python
lr = 0.001  # Örnek sabit learning rate


* Basit, küçük ağlarda genellikle yeterli

* Derin ve kompleks ağlarda bazen stabilite sorunları yaşanabilir

## 🔹 StepLR


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

NameError: name 'optimizer' is not defined

* Belirli epoch sayısı sonrası LR’yi çarpan (gamma) ile azaltır

* Örnek: Her 10 epoch’ta LR 0.1 ile çarpılır

## 🔹 ExponentialLR

In [ ]:
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

* Her epoch’ta LR üstel olarak azalır

* Kararlı ve yumuşak LR düşüşü sağlar

## 🔹 ReduceLROnPlateau !!!!!!

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

* Validasyon kaybı belirli bir süre iyileşmezse LR’yi azaltır

* Otomatik olarak LR ayarlamak için faydalı

* Özellikle overfitting veya plateau durumlarında kullanılır

| Scheduler         | Amaç                           | Notlar                                        |
| ----------------- | ------------------------------ | --------------------------------------------- |
| Sabit LR          | Basit ve küçük ağlar           | Küçük ve stabil problem setlerinde yeterli    |
| StepLR            | Belirli aralıklarla LR düşürme | Epoch tabanlı, gamma ile çarpılır             |
| ExponentialLR     | Üstel LR düşüşü                | Daha yumuşak ve stabil düşüş sağlar           |
| ReduceLROnPlateau | Plateau durumunda LR azaltma   | Validasyon kaybı iyileşmezse otomatik çalışır |


---


# 5️⃣ Label Smoothing

## 📌 Amaç
- Modelin **aşırı güvenli tahminler yapmasını** engeller  
- Cross-Entropy Loss yerine, sınıf dağılımını biraz "yumuşatır"  
- Bu sayede **genelleme** artar ve **overfitting** azalır  



## 🔹 Nasıl Çalışır?
- Normalde doğru sınıf etiketi **%100 (1.0)** oluyordu  
- Label smoothing ile → **%90 doğru sınıf, %10 diğer sınıflara dağıtılır**  
- Örneğin:  
  - Normal → `[0, 0, 1, 0]`  
  - Smoothing (ε=0.1) → `[0.033, 0.033, 0.9, 0.033]`  



## 🔹 Kod Örneği
```python
import torch
import torch.nn as nn

# Model çıkışı (örnek)
outputs = torch.randn(4, 5)  # 4 örnek, 5 sınıf
labels = torch.tensor([2, 1, 0, 3])  # gerçek etiketler

# Label smoothing'li loss
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
loss = criterion(outputs, labels)

print("Loss:", loss.item())


---

# Aşağıda bulunan modellerin bazıları örneklerde kullanılmıştır.Bu tür model kullanımlarına aşina olunuz.,


In [4]:
class ANN_MODEL(nn.Module):
    def __init__(self, input_dim , hidden_layers=[128,64,32], dropout=0.3):
        super(ANN_MODEL ,self).__init__()

        self.layers = nn.ModuleList()
        in_dim = input_dim

        for h_dim in hidden_layers:
            self.layers.append(nn.Linear(in_dim,h_dim))
            in_dim = h_dim

        self.out = nn.Linear(in_dim,1)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self , x):
        for layer in self.layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = self.out(x)

        return x 

In [5]:
class AdvancedANN(nn.Module):
    def __init__(self, input_dim, hidden_layers=[256, 128, 64, 32], dropout=0.3):
        super(AdvancedANN, self).__init__()
        self.layers = nn.ModuleList()
        self.dropout = nn.Dropout(dropout)
        in_dim = input_dim

        for h_dim in hidden_layers:
            self.layers.append(nn.Linear(in_dim, h_dim))
            in_dim = h_dim

        self.out = nn.Linear(in_dim, 1)
    
    def forward(self, x):
        residual = x  # İlk input residual bağlantısı için
        for i, layer in enumerate(self.layers):
            x = layer(x)
            x = F.relu(x)
            x = self.dropout(x)
            
            # Her 2 katmanda bir residual bağlantı ekle
            if i % 2 == 1:
                if residual.shape[1] == x.shape[1]:
                    x = x + residual
                residual = x  # residual'u güncelle
            
        x = self.out(x)
        return x


In [ ]:
class ANN(nn.Module):
    def __init__(self, input_size , output_size = 1, dp=0.3):
        super(ANN ,self).__init__()

        def linear_block(in_features , out_features , dropout):
            return nn.Sequential(

                nn.Linear(in_features , out_features),
                nn.BatchNorm1d(out_features),
                nn.LeakyReLU(0.1),
                nn.Dropout(dropout)        
                )
        
        self.model = nn.Sequential(
                linear_block(input_size , 128 , dp),
                linear_block(128,64,dp),
                linear_block(64,32,dp),
                nn.Linear(32,output_size)
            )

        for m in self.model:
            if isinstance(m,nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)
        
    def forward(self,x):
        return self.model(x)